In [2]:
import json 
with open('documents.json', 'rt') as  f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv

base_url=os.environ.get("BASE_URL")
api_key=os.environ.get("API_KEY")

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

In [5]:
def build_prompt(query, search_results):
    prompt_template = """
    Your are a course teaching assistant. 
    Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT  when answering the QUESTION. 
    If the CONTEXT doesn't containt the answer output None.

    QUESTION:
    {question}

    CONTEXT:
    {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 


In [6]:
def llm(prompt):
    responce = client.chat.completions.create(
        model="qwen/qwen3-coder:free",
        messages=[{
            "role":"user",
            "content":prompt
        }]
    )
    return responce.choices[0].message.content

In [7]:
query = "how do I run kafka?"

```bash
 docker run -it \
   --rm \
   --name elasticsearch \
   -m 4GB \
   -p 9200:9200 \
   -p 9300:9300 \
   -e "discovery.type=single-node" \
   -e "xpack.security.enabled=false" \
   elasticsearch:9.1.0

```

run elastic searrch with this docker code


In [8]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [9]:
es_client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7847cdaaaf30>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 3/948 [00:00<00:31, 29.88it/s]

100%|██████████| 948/948 [00:05<00:00, 175.64it/s]


In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
query = "how do I run kafka?"
rag(query)


'To run Kafka, you can follow these steps based on the context provided:\n\n1. **For Java-based Kafka applications (e.g., producer/consumer/kstreams):**\n   - Navigate to your project directory.\n   - Run the following command in the terminal:\n     ```\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n     *(Note: Replace `<jar_name>` with the actual name of your JAR file.)*\n\n2. **For Python-based Kafka applications (e.g., `producer.py`):**\n   - Create and activate a virtual environment:\n     ```\n     python -m venv env\n     source env/bin/activate   # On Windows: env\\Scripts\\activate\n     ```\n   - Install the required dependencies:\n     ```\n     pip install -r ../requirements.txt\n     ```\n   - Ensure Docker images (if used) are up and running before executing the Python files.\n\nThese instructions should help you run Kafka in either a Java or Python environment.'

```bash
docker pull qdrant/qdrant

docker run -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
```

start the qd_client

In [15]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient('http://localhost:6333')

EMBEDDING_DIMENSIONALITY  = 512

model_handle = "jinaai/jina-embeddings-v2-small-en"
 
# Define the collection name
collection_name = 'zoomcamp-faq'


In [16]:
qd_client.delete_collection(collection_name=collection_name)

True

In [17]:

# Create the collection with the specific vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, # Dimensionality of the vectors
        distance=models.Distance.COSINE # Distance metrics of the vectors
    )
)

True

In [18]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text'] 
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector, 
        payload=doc  
    )
    points.append(point)

In [21]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [22]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
def vector_search(query):
    print('Vector Seaach is Used')
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(  #embedded the query text locally with jinaai
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )

    results = []

    for point in query_points.points:
        results.append(point.payload)
    
    return results 

In [28]:
search_results = vector_search(query)


Vector Seaach is Used


In [29]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
query = "how do I run kafka?"
rag(query)

Vector Seaach is Used


'To run Kafka, follow these steps based on the context provided:\n\n1. **For Java-based Kafka components (e.g., producer, consumer, KStreams):**\n   - Navigate to your project directory.\n   - Run the following command in the terminal:\n     ```\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n     Replace `JsonProducer.java` with the specific component you want to run (e.g., `JsonConsumer.java`).\n\n2. **Ensure correct configurations:**\n   - Verify that the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` in your Java scripts points to the correct server URL.\n   - Update the Kafka cluster key and secret in `src/main/java/org/example/Secrets.java`.\n\n3. **If using Docker for Kafka brokers:**\n   - If encountering a `NoBrokersAvailable` error, check if your Kafka broker Docker container is running:\n     ```\n     docker ps\n     ```\n   - If not running, start the Docker containers using:\n     ```\n     docker compose up -d\n 